In [ ]:
!pip install category_encoders

In [15]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians, log, log1p
import random
from time import gmtime, strftime
import copy
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from category_encoders import TargetEncoder # bonus: sklearn-contrib
from sklearn.metrics import roc_auc_score
import requests

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [16]:
def read_csv_from_url(url, file_name):
    """Обучающая и тестовая выборки предварительно загружены на яндекс-диск
    Функция выполняет скачивание и загрузку в датафрейм csv-файла по заданной ссылке"""
    r = requests.get(url, allow_redirects=True)
    open(file_name, 'wb').write(r.content)
    return pd.read_csv(file_name)

In [17]:
train = read_csv_from_url('https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/BXKOOeLfLQvfqw', 'fraudTrain.csv')
test  = read_csv_from_url('https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/uL7gLcpQkJLUsw', 'fraudTest.csv')

In [106]:
train.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [70]:
def calc_distance(lat1, long1, lat2, long2):
    """Определение дистанции между точками"""
    R = 6373.0 # примерный радиус Земли в км
    lat1 = radians(lat1)
    lon1 = radians(long1)
    lat2 = radians(lat2)
    lon2 = radians(long2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [71]:
def preprocess_features_basic(df):
    """Функция добавляет к датафрейму аттрибуты:
    - trans_hour     - час транзакции;
    - trans_week_day - день недели транзакции;
    - birth_year     - год рождения плательщика;
    - trans_distance - дистанция между местом жительства плательщика и местом транзакции;
    - trans_hex_id   - пространственный индекс места совершения операции (см. https://eng.uber.com/h3/)
    """
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Preprocess features: trans_hour, trans_dttm, dob, trans_week_day, trans_hex_id')
    df['trans_hour'] = df.loc[:,'trans_date_trans_time'].str[11:13]
    df['trans_dttm'] = pd.to_datetime(df.loc[:,'trans_date_trans_time'])
    df['birth_year'] = df.loc[:,'dob'].str[0:4].astype('int')
    df['trans_week_day'] = df.loc[:,'trans_dttm'].apply(lambda x: x.strftime('%A'))
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Preprocess features: trans_distance')
    df['trans_distance'] = df.apply(lambda row: calc_distance(row["lat"], row["long"], row["merch_lat"], row["merch_long"]), axis = 1)
    #df['trans_hex_id'] = df.apply(lambda row: h3.geo_to_h3(row["merch_lat"], row["merch_long"], resolution = 3), axis = 1)
    return df

In [72]:
def target_share_by_feature(feature_name, target_name, df):
    """Функция рассчитывает характеристику встречаемости фрода для каждого значения
    категориального аттрибута.
    Возвращает датафрейм из 3х колонок: наименование аттрибута, количество единичек, долю единичек.
    """
    agg_df = df.groupby([feature_name,target_name], as_index=False)\
            .aggregate({'unix_time' : 'count'})\
            .rename(columns = {'unix_time': 'cnt'})
    pivot_df = agg_df.pivot(index=feature_name, columns=target_name).fillna(0)
    flat_df = pd.DataFrame(pivot_df.to_records())
    flat_df.columns = [feature_name, 'target_0', 'target_1']
    flat_df['target_share'] = flat_df['target_1'] / (flat_df['target_0'] + flat_df['target_1'])
    flat_df.sort_values(by='target_share', ascending=False, inplace = True)
    flat_df.columns = [feature_name, feature_name + '_target_0_cnt', feature_name + '_target_1_cnt', feature_name + '_target_share']
    flat_df = flat_df.drop(feature_name + '_target_0_cnt', axis = 1)
    return flat_df

In [73]:
def get_typical_zip_ref(train, min_typical_trans_cnt = 400):
    """Создадим справочник типовых локаций"""
    zip_df = train.loc[train['is_fraud']==0,:].groupby(['zip','is_fraud'], as_index=False)\
            .aggregate({'unix_time' : 'count'})\
            .rename(columns = {'unix_time': 'cnt'})
    typical_zip = list(zip_df.loc[zip_df['cnt']>min_typical_trans_cnt,:]['zip'])
    return typical_zip

In [74]:
def get_typical_job_ref(train, min_typical_trans_cnt = 400):
    """Создадим справочник типовых должностей"""
    job_df = train.loc[train['is_fraud']==0,:].groupby(['job','is_fraud'], as_index=False)\
            .aggregate({'unix_time' : 'count'})\
            .rename(columns = {'unix_time': 'cnt'})
    typical_job = list(job_df.loc[job_df['cnt']>min_typical_trans_cnt,:]['job'])
    return typical_job

In [75]:
def mark_typical_features(train, test = None, type_cd = 'train'):
    """Добавим фичи:
    - typical_zip - признак типовой локации
    - typical_job - признак типовой должности
    """
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Preprocess features: mark typical_zip, typical_job')
    typical_zip_ref = get_typical_zip_ref(train)
    typical_job_ref = get_typical_job_ref(train)
    if type_cd == 'train':
        df = train
    if type_cd == 'test':
        df = test
    df['typical_zip'] = df['zip'].isin(typical_zip_ref).astype(int)
    df['typical_job'] = df['job'].isin(typical_job_ref).astype(int)
    return df

In [76]:
def mark_earlier_fraudsters(train, test = None, type_cd = 'train'):
    """Разметим карточки, которые ранее были уличены во фроде
    Функция добавляет к датафрейму колонку earlier_fraudster со значениями 1 или 0
    В train обязательно устраняем лик, когда разметка берется из будущего"""
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Preprocess features: mark earlier fraudsters')
    fraudster_list = list(set(train.loc[train['is_fraud']==1,'cc_num']))
    if type_cd == 'train':
        first_fraud_dttm = train.loc[train['is_fraud']==1,:]\
        .groupby(['cc_num','is_fraud'], as_index=False)\
        .aggregate({'unix_time' : 'min'})
        first_fraud_dttm['first_fraud_flag']=1
        train = train.merge(first_fraud_dttm, on=['cc_num','unix_time'], how='left')\
                     .rename(columns = {'is_fraud_x':'is_fraud'})
        train['earlier_fraudster'] = train['cc_num'].isin(fraudster_list).astype(int)
        train.loc[train['first_fraud_flag']==1,'earlier_fraudster'] = 0
        train.drop(['is_fraud_y','first_fraud_flag'], axis = 1, inplace = True)
        return train
    if type_cd == 'test':
        test['earlier_fraudster'] = test['cc_num'].isin(fraudster_list).astype(int)
        return test

In [77]:
def get_first_transaction_time(train, test = None, type_cd = 'train'):
    """Определим время первой транзакции для каждой карточки"""
    if type_cd == 'train':
        cc_num_min_time = train.groupby(['cc_num'], as_index=False)\
                .aggregate({'unix_time' : 'min'})\
                .rename(columns = {'unix_time': 'first_trans_time'})
        return cc_num_min_time
    if type_cd == 'test':
        frames = [train, test]
        train_test_concat = pd.concat(frames)
        cc_num_min_time = train_test_concat.groupby(['cc_num'], as_index=False)\
                .aggregate({'unix_time' : 'min'})\
                .rename(columns = {'unix_time': 'first_trans_time'})
        return cc_num_min_time

In [78]:
def mark_life_time_days(train, test = None, type_cd = 'train'):
    """Посчитаем время жизни карточки с момента первой транзакции"""
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Preprocess features: calc life time days')
    cc_num_min_time = get_first_transaction_time(train, test, type_cd)
    if type_cd == 'train':
        df = train
    if type_cd == 'test':
        df = test
    df = df.merge(cc_num_min_time, on='cc_num', how='left')
    df['life_time_days'] = df['unix_time'] - df['first_trans_time']
    df['life_time_days'].fillna(0, inplace = True)
    df['life_time_days'] = df['life_time_days'] / (60*60*24)
    df['life_time_days'] = df['life_time_days'].apply(lambda x: x + random.uniform(0.05,0.5))
    df['life_time_days'] = df['life_time_days'].round(3)
    return df

In [79]:
def mark_merchant_fraud_share(train, test = None, type_cd = 'train'):
    """Определим долю фрода по каждому продавцу"""
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Preprocess features: mark merchant fraud share')
    merchant_fraud_share = target_share_by_feature('merchant', 'is_fraud', train)
    merchant_fraud_share.drop('merchant_target_1_cnt', axis = 1, inplace = True)
    if type_cd == 'train':
        df = train
    if type_cd == 'test':
        df = test
    df = df.merge(merchant_fraud_share, on='merchant', how='left')
    df['merchant_target_share'].fillna(0, inplace = True)
    return df

In [80]:
def delete_unwanted_features(df):
    """ Удалим лишние аттрибуты """
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Deleting unwanted features')
    features_to_delete = ['Unnamed: 0','trans_date_trans_time','cc_num','merchant',
                          'first','last','street','city','state','zip','lat','long',
                          'job','dob','trans_num','unix_time','merch_lat','merch_long',
                          'trans_dttm','first_trans_time']
    df.drop(features_to_delete, axis = 1, inplace = True)
    return df

In [81]:
def z_scale(feature_name_list, train, test = None, type_cd = 'train'):
    """Функция выполняет Z-масштабирование аттрибутов: (x - mean) / standard_deviation
    """
    # Standardscaler - есть стандатная функция.
    for feature_name in list(feature_name_list):
        print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Z scale: ' + feature_name)
        mean_x = train[feature_name].mean()
        std_x = train[feature_name].std()
        if type_cd == 'train':
            df = train
        if type_cd == 'test':
            df = test
        df[feature_name+'_stdnorm'] = df[feature_name].apply(lambda x: (x - mean_x) / (std_x))
        df = df.drop(feature_name, axis = 1)
    return df

In [82]:
def min_max_scale(df, feature_name_list):
    """ Функция выполняет min-max нормализацию аттрибутов: (x - min_x) / (max_x-min_x)
    """
    # MinMaxScaler (стандартная функция sklearn)
    for feature_name in list(feature_name_list):
        print(strftime("%Y-%m-%d %H:%M:%S", gmtime()) + ' Min_max_scale: ' + feature_name)
        min_x = df[feature_name].min()
        max_x = df[feature_name].max()
        df[feature_name+'_norm'] = df[feature_name].apply(lambda x: (x - min_x) / (max_x-min_x))
        df = df.drop(feature_name, axis = 1)
    return df

In [83]:
train = preprocess_features_basic(train)
train = mark_life_time_days(train)
train_init = copy.deepcopy(train)
test_init = copy.deepcopy(test)

2020-11-26 23:46:47 Preprocess features: trans_hour, trans_dttm, dob, trans_week_day, trans_hex_id
2020-11-26 23:47:08 Preprocess features: trans_distance
2020-11-26 23:49:32 Preprocess features: calc life time days


In [84]:
train = mark_typical_features(train)
train = mark_earlier_fraudsters(train)
train = mark_merchant_fraud_share(train)
train = delete_unwanted_features(train)

2020-11-26 23:49:42 Preprocess features: mark typical_zip, typical_job
2020-11-26 23:49:44 Preprocess features: mark earlier fraudsters
2020-11-26 23:49:54 Preprocess features: mark merchant fraud share
2020-11-26 23:50:01 Deleting unwanted features


In [85]:
features_to_scale = ['amt','city_pop', 'trans_distance', 'life_time_days', 'birth_year']
train = z_scale(features_to_scale, train)

2020-11-26 23:50:02 Z scale: amt
2020-11-26 23:50:03 Z scale: city_pop
2020-11-26 23:50:05 Z scale: trans_distance
2020-11-26 23:50:06 Z scale: life_time_days
2020-11-26 23:50:07 Z scale: birth_year


In [86]:
test = preprocess_features_basic(test)
test = mark_life_time_days(train = train_init, test = test, type_cd = 'test')
test = mark_typical_features(train = train_init, test = test, type_cd = 'test')
test = mark_earlier_fraudsters(train = train_init, test = test, type_cd = 'test')
test = mark_merchant_fraud_share(train = train_init, test = test, type_cd = 'test')
test = delete_unwanted_features(test)
test = z_scale(features_to_scale, train = train_init, test = test, type_cd = 'test')

2020-11-26 23:50:08 Preprocess features: trans_hour, trans_dttm, dob, trans_week_day, trans_hex_id
2020-11-26 23:50:17 Preprocess features: trans_distance
2020-11-26 23:51:19 Preprocess features: calc life time days
2020-11-26 23:51:24 Preprocess features: mark typical_zip, typical_job
2020-11-26 23:51:26 Preprocess features: mark earlier fraudsters
2020-11-26 23:51:26 Preprocess features: mark merchant fraud share
2020-11-26 23:51:27 Deleting unwanted features
2020-11-26 23:51:28 Z scale: amt
2020-11-26 23:51:28 Z scale: city_pop
2020-11-26 23:51:28 Z scale: trans_distance
2020-11-26 23:51:29 Z scale: life_time_days
2020-11-26 23:51:29 Z scale: birth_year


# Обучение модели

In [91]:
target_encoder = TargetEncoder(cols=train.select_dtypes(include=[object]).columns)
estimator = XGBClassifier(nthread=-1, n_estimators=1000, learning_rate=0.1)

pipeline = Pipeline([
    ('my_encoder', target_encoder),
    ('my_estimator', estimator)
])

In [92]:
X, X_test, y, y_test = train_test_split(train, train['is_fraud'],
                                         stratify=train['is_fraud'],
                                         test_size=0.3,
                                         random_state=43)

# отделяем от данных для обучения небольшой сэмпл для early_stopping
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                 stratify=y,
                                                 test_size=0.1,
                                                 random_state=43)

# итого
X_train.shape, X_val.shape, X_test.shape

((816904, 18), (90768, 18), (389003, 18))

In [93]:
X_train = X_train.drop(['is_fraud'], axis=1)
X_val = X_val.drop(['is_fraud'], axis=1)
X_test = X_test.drop(['is_fraud'], axis=1)

In [94]:
from sklearn.base import clone

initial_steps = []
for i in pipeline.steps:
    initial_steps.append(clone(i[1]))

transformers_pipeline = make_pipeline(*initial_steps[:-1])
transformers_pipeline.fit(X_train, y_train)

X_val_transformed = transformers_pipeline.transform(X_val)

In [95]:
# передаем в fit параметры для early_stopping как: my_estimator__eval_set
# учится очень быстро -  смысл в том, что берутся случайным образом перемешанные данные
# высокий градиент функции потерь
pipeline.fit(X_train, y_train,
            my_estimator__eval_set=[(X_val_transformed, y_val)], 
            my_estimator__eval_metric='auc', 
            my_estimator__early_stopping_rounds=5, 
            my_estimator__verbose=10)

[0]	validation_0-auc:0.96039
Will train until validation_0-auc hasn't improved in 5 rounds.
[10]	validation_0-auc:0.96604
Stopping. Best iteration:
[11]	validation_0-auc:0.96621



Pipeline(memory=None,
         steps=[('my_encoder',
                 TargetEncoder(cols=Index(['category', 'gender', 'trans_hour', 'trans_week_day'], dtype='object'),
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', min_samples_leaf=1,
                               return_df=True, smoothing=1.0, verbose=0)),
                ('my_estimator',
                 XGBClassifier(base_score=0.5, booster=None,
                               colsample_bylevel=1, colsample_...
                               interaction_constraints=None, learning_rate=0.1,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=nan,
                               monotone_constraints=None, n_estimators=1000,
                               n_jobs=-1, nthread=-1, num_parallel_tree=1,
                               objective='binary:logistic', random_state=0,
           

In [96]:
y_pred = pipeline.predict_proba(X_test)
roc_auc_score(y_test, y_pred[:, 1])

0.9673787545745959

# Test results

In [99]:
X_test_real = test.drop('is_fraud', axis = 1)
y_test_real = test.loc[:,'is_fraud']

In [100]:
y_pred = pipeline.predict_proba(X_test_real)
roc_auc_score(y_test_real, y_pred[:, 1])

0.960994441687899

In [101]:
data_tuples = list(zip(y_test_real,list(y_pred[:,1])))
compare = pd.DataFrame(data_tuples, columns=['y_true','y_probas'])
compare['y_probas'] = compare['y_probas'].round(3)

In [ ]:
compare.to_excel('compare_8.xlsx', index = False)

# Feature importance

In [102]:
fi=pd.DataFrame({'importance':pipeline[1].feature_importances_},index=X_train.columns)
fi.sort_values('importance',ascending=False).head(15)
# берем фичу, по ней упорядочиваем весь датасет. Например отсортировали по возрасту. Смотрим как поверхность делит выборку и на какие состояния
# например больши и меньше 30 лет. На этих областях считается empurity из пропорций
# всего доля единичек - 1%. Меньше 30 - 0.5% а больше 30 - 0.5%
# Критерий Джини: - 1 - вероятность 1 класса и 1 - вероятность 0 класса. 
# При каждом делении неопределенность должна уменьшаться. 1 минус сумма квадратов по одному классу и 1 минус сумма квадратов по другому классу. 
# Когда по какой-то фиче делим - получаем информационный прирост.
# featureimportance - информационный прирост, полученный для каждой фичи. какая-то фича может фигурировать в нескольких уровнях дерева. 
# featureimportance - пример с шахматной доской (делим несколько раз вдоль и поперек)

,importance
typical_zip,0.219558
amt,0.201036
trans_hour,0.191122
category,0.151919
gender,0.074960
birth_year_stdnorm,0.061552
earlier_fraudster,0.044733
merchant_target_share,0.031121
city_pop,0.015540
trans_week_day,0.004728


In [ ]:
# model = XGBClassifier(nthread=-1, max_depth = 4, learning_rate=0.1, n_estimators=100, min_child_weight = 5, reg_alpha=0.01, reg_lambda = 0.1)
# alpha gamma - регуляризация модели, чтобы избежать переобучения
# для лог.регрессии - числа больше 0. Чем меньше число, тем больше регуляризация. Обычно от 0.0001 до 1000. 
# Регуляризация подстраивается под конкретные данные. 
# чем меньше значение, тем проще модель (в случае с логистической регрессией) чем больше лябда, тем больше регуляризация
# Задача увеличить коэффициент регуляризации. 
# Бустинг: использует слабые эстиматоры (оценщики), которые идут друг за другом подряд. Получаем ошибку и даем на вход 
# Эстиматор - это дерево решений. (c набором параметров)
# используется дерево решений с глубиной 6 по умолчанию 
# n_estimators - количество эстиматоров, идущих друг за другом
# в каждом эстиматоре есть глубина - уменьшить глубину. max_depth - уменьшить (по умолчанию должно стоять 6)
#reg_alpha - коэффициент регуляризации
#reg_lambda - коэффициент регуляризации
# learning_rate - характеризует скорость движения к минимуму функции потерь. Чем он меньше, тем медленнее и аккуратнее мы движемся.
# при малых значениях скорее всего получим недообучение, т.к. наша функция потерь не будет меняться. 
# Идеальная статья про параметны xgboost http://biostat-r.blogspot.com/2016/08/xgboost.html

In [ ]:
# Варианты развития модели:
# - Попробовать сбалансировать классы
# - undersampling (проверить это в sklearn и imblearn)
# - Попробовать oversampling
# - Попробовать синтетическое добавление (imblearn - класс SMOTE)
# - adasyn - adaptive syntetic generation - разбавить синтетическими данными, но мерить качество (валидироваться) без синтетических данных
# - Для избежания переобучения - сократить количество фичей
# - Регуляризация - добавление штрафа в функцию потерь. (L1 & L2 - регуляризация)